# Machine Learning Project

## Introduction

After taking the Data Science course from BrainStation, I learned how powerful python and its libraries in data analysis, prediction and visualization. In order to strengen my skills in algorithm development, I decided to take a step further to continue and deep dive into Machine Learning. 

Currently I am at the 3rd week of the course and planning on my project, it took me a while to decide a topic as I tried to avoid having repetitive skills as used in Data Science course. I learned that there are various stages of machin learning:
    1. Data collection
    2. Data sorting
    3. Data analysis
    4. Algorithm development
    5. Apply and optimize algorithm
    6. Utilize the results geneterated from the algorithm to provide insights and further conclusions

In order to apply all the above skills and use all sort of machine learning APIs more extensively including sci-kit learn, TensorFlow and Keras, I have done research and read feedback on suggested projects, and decided to proceed with analyzing Real and Fake news. Data was taken from Kaggle and the project is in progress. I will constantly upload the noteboook for updates, so stay tuned!

In [ ]:
# import libraries
# more libraries will be imported as I go

# libraries for data processing and data analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

%matplotlib inline

# visual representation of text data
from wordcloud import WordCloud

# libraries for machine learning
